# Eight queens puzzle - using Genetic algorithm
**Karthikeyan Ramachandran**

## About
The eight queens puzzle is the problem of placing eight chess queens on an 8×8 chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. 

## Steps involved
1. Initial population is generated with chromosomes containing random genes.
2. Fitness score of the population is calculated.
3. Based on fitness score, parents are selected for mating.
4. Crossover and Mutation are performed on the selected parents, to produce offsprings.
5. Offsprings are added to the population and Fitness score is calculated.
6. Process is stopped if max-fitness score is found, else repeated from Step 3 to 5

## Import libraries

In [142]:
import numpy as np
import pandas as pd
import unittest

# Setting seed value for reproducibility
np.random.seed(50)

## Define parameters

In [143]:
# Define sizes of initial-population, children and generations 
initial_population_size = 100
num_of_children = 50 # should be even number
num_of_generation = 500

# position to perform Crossover operation on the combination
crossOver=0.5

# Define fitness parameters
num_of_queens = 8
num_of_checks = 7
target_value = num_of_queens * num_of_checks

## Define functions

In [144]:
# Generate initial-population
def generate_init_pop(pop_size):
    initial_population = []
    for i in range(pop_size):
        initial_population.append(np.random.randint(0,8,8))
    return initial_population

In [145]:
# Fitness score-dependent functions

## Get attack positions
def get_atk_position(pos, dist):
    atk_position = np.array([pos-dist, pos, pos+dist], dtype="int32")
    return atk_position

## Get relative position of the queen (to calculate distance)
def get_rel_pos(position_array, idx_val):
    rel_pos = np.empty(8)
    for i in range(len(position_array)):
        if idx_val < i:
            rel_pos[i] = (i - idx_val)
        else:
            rel_pos[i] = (idx_val - i)
            
    return rel_pos

In [146]:
# Fitness score 

## Max fitness score is 8 queens * 7 attacks from other queens = 56
def fitness_score(position_array):
    score = 0
    for i in range(len(position_array)):
        rel_pos = get_rel_pos(position_array, i)
        for j in range(len(position_array)):
            if i == j:
                continue
            atk_position = get_atk_position(position_array[i], rel_pos[j])
            if position_array[j] in atk_position:
                is_attack = True
            else:
                is_attack = False
                score += 1
            # print(position_array[j], atk_position, is_attack, score)
    return score

In [147]:
# generate probabilty based on fitness score
def get_probability(fitness_value, target_value = 56):
    probability = np.empty(len(fitness_value))
    for i in range(len(fitness_value)):
        probability[i] = (np.round(fitness_value[i]/target_value, 5))
    return probability

In [148]:
# sort dataframe based on probability
def prob_sort(population_dataframe):
    population_dataframe = (population_dataframe.sort_values(["probability"], ascending=False).reset_index(drop=True))
    return population_dataframe

In [149]:
# store generated info into a dataframe
def get_dataframe(combination_array):
    population = []
    fitness = []

    for i in range(len(combination_array)):
        combination = combination_array[i]
        population.append(combination)
        fitness.append(fitness_score(combination))

    pop_df = pd.DataFrame({"combination":population, "fitness":fitness})
    pop_df["probability"] = get_probability(pop_df["fitness"])
    pop_df = prob_sort(pop_df)

    return pop_df

In [150]:
# function to perform One Point Crossover
def crossover(parent1, parent2, crossOver=0.5):
    # get Crossover length to slice
    crossOver_len = round(int(len(parent1) * crossOver))
    
    # Slice and perform crossover
    child1 = np.concatenate((parent1[:crossOver_len], parent2[crossOver_len:]), axis=0)
    child2 = np.concatenate((parent2[:crossOver_len], parent1[crossOver_len:]), axis=0)
    
    return child1, child2

In [151]:
# function to perform Random Resetting mutation
def mutation(combination):
    random_queen = np.random.randint(8)
    random_position = int(np.random.randint(0,8,1))
    combination[random_queen] = random_position
    return combination

In [152]:
# generate offspring combination based on performing Crossover and Mutation on selected parents
def get_children(parent_array):

    children = []
    mutated_children = []

    for i in range(0,num_of_children,2):
        p1 = parent_array[i]
        p2 = parent_array[i+1]
    
        (c1, c2) = crossover(p1, p2)
    
        children.append(c1)
        children.append(c2)
    
    for i in range(num_of_children):
        mutated_children.append(mutation(children[i]))
    
    return mutated_children

In [153]:
# Function to generate the right sequence
def generate_sequence(initial_population, children, generations, expected_fitness):
    # get initial population fitness

    init_pop_fitness = get_dataframe(initial_population)
    mating_pool = init_pop_fitness[:children]

    print("expected fitness:", expected_fitness, "\n----------------------------------n")

    for i in range(generations):
        # get expected fitness count
        expected_fitness_count = mating_pool[(mating_pool["fitness"]==expected_fitness)]["fitness"].count()
        if expected_fitness_count > 0:
            print("Gen:", (i-1), 
                ", best-combination:", mating_pool["combination"][0], 
                ", fitness:", mating_pool["fitness"][0],
                 ", probability:", mating_pool["probability"][0]
             )
            break
        gen_pop = []
        gen_pop = get_children(mating_pool["combination"])
        gen_pop = np.concatenate((gen_pop, list(mating_pool["combination"])), axis=0)
        mating_pool = get_dataframe(gen_pop)
    
        print("Gen:", i, 
            ", best-combination:", mating_pool["combination"][0], 
            # mating_pool["combination"][0], mating_pool["combination"][1], mating_pool["combination"][2],
            ", fitness:", mating_pool["fitness"][0],
            ", avg-fitness:", np.round(np.mean(mating_pool["fitness"]), 3),
            ", probability:", mating_pool["probability"][0]
            )
    return mating_pool[mating_pool["probability"]==1]["combination"][0]

## Unit testing

### Define Unit test cases

In [154]:
class TestQueens(unittest.TestCase):
    '''
    Unit test cases to validate functions used in the solution
    '''

    # Test the function to generate initial population
    def test_generate_init_pop(self):
        test_init_pop = generate_init_pop(3)
        self.assertEqual(len(test_init_pop), 3) # validate length

    # Test the Queen's attack positions
    def test_get_atk_position(self):
        test_atk_pos = get_atk_position(5, 2)
        valid_atk_pos = np.array([3, 5, 7], dtype="int32")
        self.assertSequenceEqual(test_atk_pos.tolist(), valid_atk_pos.tolist())

    # Test the relative position array
    def test_get_rel_pos(self):
        test_rel_pos = get_rel_pos(np.arange(8), 3)
        valid_rel_pos = np.array([3, 2, 1, 0, 1, 2, 3, 4], dtype="float")
        self.assertSequenceEqual(test_rel_pos.tolist(), valid_rel_pos.tolist())

    # Test the fitness score funtion
    def test_fitness_score(self):
        test_fit_all_eight = fitness_score(np.full(8, 8))
        test_fit_diag1 = fitness_score(np.arange(8))
        test_fit_diag1 = fitness_score(np.arange(7, -1, -1))
        test_fit_best = fitness_score(np.array([4,0,7,3,1,6,2,5]))
        test_fit_sample = fitness_score(np.array([0,1,5,7,2,6,3,6]))

        self.assertEqual(test_fit_all_eight, 0)
        self.assertEqual(test_fit_diag1, 0)
        self.assertEqual(test_fit_diag1, 0)
        self.assertEqual(test_fit_best, 56)
        self.assertEqual(test_fit_sample, 52)

    # Test probability function
    def test_get_probability(self):
        test_prob = get_probability(np.array([56, 50, 48]), 56).tolist()
        valid_prob = np.array([1., 0.89286, 0.85714]).tolist()
        self.assertSequenceEqual(test_prob, valid_prob)

    # Test get dataframe
    def test_get_dataframe(self):
        test_df = get_dataframe([np.full(8, 8, dtype="int32")])
        test_df_combination = test_df["combination"][0].tolist()
        test_df_probability = test_df["probability"][0]
        val_combination = np.full(8, 8, dtype="int32").tolist()
        val_probability = 0.0
        self.assertSequenceEqual(test_df_combination, val_combination)
        self.assertEqual(test_df_probability, val_probability)

    # Test crossover
    def test_crossover(self):
        p1 = np.full(8, 8); p2 = np.full(8, 0)
        c1, c2 = crossover(p1, p2, crossOver=0.5)
        val_c1 = np.array([8, 8, 8, 8, 0, 0, 0, 0])
        val_c2 = np.array([0, 0, 0, 0, 8, 8, 8, 8])
        self.assertSequenceEqual(c1.tolist(), val_c1.tolist())
        self.assertSequenceEqual(c2.tolist(), val_c2.tolist())


### Validate Unit test cases

In [155]:
if __name__ == "__main__":
    unittest.main(argv=["None"], exit=False)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.023s

OK


## Code execution

### Generate inital-population

In [156]:
# Generate initial-population
init_pop = generate_init_pop(initial_population_size)

# display initial-population
init_pop[:5]

[array([2, 6, 4, 3, 7, 1, 5, 2]),
 array([1, 0, 6, 3, 6, 3, 2, 7]),
 array([6, 1, 3, 3, 2, 4, 3, 2]),
 array([4, 0, 3, 2, 2, 0, 6, 3]),
 array([0, 5, 5, 6, 0, 3, 6, 2])]

### Generate right sequence

In [157]:
# store right sequence
right_sequence = generate_sequence(init_pop, num_of_children, num_of_generation, target_value)

expected fitness: 56 
----------------------------------n
Gen: 0 , best-combination: [6 5 3 1 4 7 5 0] , fitness: 52 , avg-fitness: 42.86 , probability: 0.92857
Gen: 1 , best-combination: [6 5 3 1 4 7 5 0] , fitness: 52 , avg-fitness: 42.64 , probability: 0.92857
Gen: 2 , best-combination: [6 5 3 1 4 7 5 0] , fitness: 52 , avg-fitness: 42.65 , probability: 0.92857
Gen: 3 , best-combination: [6 5 3 1 4 7 5 0] , fitness: 52 , avg-fitness: 42.336 , probability: 0.92857
Gen: 4 , best-combination: [6 5 3 1 4 7 5 0] , fitness: 52 , avg-fitness: 42.593 , probability: 0.92857
Gen: 5 , best-combination: [6 1 3 7 2 4 2 0] , fitness: 52 , avg-fitness: 42.771 , probability: 0.92857
Gen: 6 , best-combination: [6 1 3 7 2 4 2 0] , fitness: 52 , avg-fitness: 43.015 , probability: 0.92857
Gen: 7 , best-combination: [6 1 3 0 4 7 5 0] , fitness: 52 , avg-fitness: 43.191 , probability: 0.92857
Gen: 8 , best-combination: [1 3 7 0 2 0 5 3] , fitness: 52 , avg-fitness: 43.396 , probability: 0.92857
Gen: 9 , 

### Display right sequence

In [177]:
# Display the right sequence
right_sequence_str = " ".join([str(i) for i in right_sequence.tolist()])
print("Sequence :", right_sequence_str)

Sequence : 5 2 4 7 0 3 1 6
